In [1]:
from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import collections
import pickle
import argparse

from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from nltk.tokenize import word_tokenize
from tensorflow.contrib import rnn
import nltk
nltk.download('punkt')

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (6,6)
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing import sequence
import os
print(os.listdir("../"))

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['ubuntu']


Using TensorFlow backend.


In [2]:
data = pd.read_json('~/test_data.json', lines=True)
data.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [4]:
cates = data.groupby('category')
print("There are:", cates.ngroups, "category")
print(cates.size())

There are: 41 category
category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       4528
BUSINESS           5937
COLLEGE            1144
COMEDY             5175
CRIME              3405
CULTURE & ARTS     1030
DIVORCE            3426
EDUCATION          1004
ENTERTAINMENT     16058
ENVIRONMENT        1323
FIFTY              1401
FOOD & DRINK       6226
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
HOME & LIVING      4195
IMPACT             3459
LATINO VOICES      1129
MEDIA              2815
MONEY              1707
PARENTING          8677
PARENTS            3955
POLITICS          32739
QUEER VOICES       6314
RELIGION           2556
SCIENCE            2178
SPORTS             4884
STYLE              2254
STYLE & BEAUTY     9649
TASTE              2096
TECH               2082
THE WORLDPOST      3664
TRAVEL             9887
WEDDINGS           3651
WEIRD NEWS         2670
WELLNESS          17827
WOMEN              3490
WORLD NEWS         2177
WORLDPOS

In [5]:
data.category = data.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)

In [6]:

# Change the text to tokens.
data['text'] = data.headline + " " + data.short_description
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.text)
X = tokenizer.texts_to_sequences(data.text)
data['words'] = X

data['word_length'] = data.words.apply(lambda i: len(i))
data = data[data.word_length >= 5]

data.head()

,authors,category,date,headline,link,short_description,text,words,word_length
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,"[74, 101, 257, 1331, 3001, 6, 698, 134, 96, 26...",27
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,"[42, 1604, 2960, 27762, 5, 25929, 5237, 8, 1, ...",20
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...,"[5877, 5334, 8083, 8, 1, 76, 54, 21, 414, 8469...",25
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,"[2710, 13374, 3596, 64143, 2295, 13055, 5, 569...",26
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...,"[41003, 36082, 1513, 97, 48, 7915, 3134, 2, 96...",26


In [7]:
data.word_length.describe()

count    199914.000000
mean         29.725032
std          14.024717
min           5.000000
25%          20.000000
50%          29.000000
75%          36.000000
max         248.000000
Name: word_length, dtype: float64

In [8]:
maxlen = 50
X = list(sequence.pad_sequences(data.words, maxlen=maxlen))

In [9]:
categories = data.groupby('category').size().index.tolist()
category_int = {}
int_category = {}
# build category to int dict and int to category dict.
for i, k in enumerate(categories):
    int_category.update({i:k})
    category_int.update({k:i})
                                    
data['c2id'] = data['category'].apply(lambda x: category_int[x])

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
data.head()

,authors,category,date,headline,link,short_description,text,words,word_length,c2id
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,"[74, 101, 257, 1331, 3001, 6, 698, 134, 96, 26...",27,6
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,"[42, 1604, 2960, 27762, 5, 25929, 5237, 8, 1, ...",20,10
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...,"[5877, 5334, 8083, 8, 1, 76, 54, 21, 414, 8469...",25,10
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,"[2710, 13374, 3596, 64143, 2295, 13055, 5, 569...",26,10
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...,"[41003, 36082, 1513, 97, 48, 7915, 3134, 2, 96...",26,10


In [11]:
def clean_setence(text):
    text = re.sub(r"[^A-Za-z0-9(),!?\'\`\"]", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = text.strip().lower()

    return text

In [12]:

texts = data["text"]

words = list()
for test in texts:
    for word in word_tokenize(clean_setence(test)):
        words.append(word)

word_counter = collections.Counter(words).most_common()
word_dictionary = dict()
word_dictionary["<padding>"] = 0
word_dictionary["<unk>"] = 1
for word, _ in word_counter:
    word_dictionary[word] = len(word_dictionary)

with open("word_dict.pickle", "wb") as f:
    pickle.dump(word_dictionary, f)

reversed_dictionary = dict(zip(word_dictionary.values(), word_dictionary.keys()))
print(len(reversed_dictionary))
doc_max_len = 20

93757


In [14]:
print("Split data...")

Split data...


In [15]:
x = list(map(lambda d: word_tokenize(clean_setence(d)), data["text"]))
x = list(map(lambda d: list(map(lambda w: word_dictionary.get(w, word_dictionary["<unk>"]), d)), x))
x = list(map(lambda d: d[:doc_max_len], x))
x = list(map(lambda d: d + (doc_max_len - len(d)) * [word_dictionary["<padding>"]], x))

y = list(data["c2id"])

In [16]:
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.10)

In [17]:
print("train and test set are prepared.")

train and test set are prepared.


In [18]:
class BidirectionalRNN(object):
    def __init__(self, reversed_dictionary, doc_max_len, num_class):
        self.x = tf.placeholder(tf.int32, [None, doc_max_len], name="x")
        self.x_len = tf.reduce_sum(tf.sign(self.x), 1)
        self.y = tf.placeholder(tf.int32, [None], name="y")
        self.keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
        self.global_step = tf.Variable(0, trainable=False)
        
        print("Run BidirectionalRNN...")
        self.vocabulary_size = len(reversed_dictionary)
        self.embedding_size = 100
        self.num_hidden = 100
        self.num_layers = 2
        self.learning_rate = 1e-3
        
        print("Loading Glove vectors...")
        glove_file = "glove.6B.%dd.txt" % self.embedding_size
        word2vec_file = get_tmpfile("word2vec_format.vec")
        glove2word2vec(glove_file, word2vec_file)
        word_vectors = KeyedVectors.load_word2vec_format(word2vec_file)

        word_vec_list = list()
        for _, word in sorted(reversed_dictionary.items()):
            try:
                word_vec = word_vectors.word_vec(word)
            except KeyError:
                word_vec = np.zeros([self.embedding_size], dtype=np.float32)

            word_vec_list.append(word_vec)


        print("start embedding...")
        init_embeddings = tf.constant(np.array(word_vec_list), dtype=tf.float32)
        self.embeddings1 = tf.get_variable("embeddings1", initializer=init_embeddings, trainable=True)
        self.x_emb = tf.nn.embedding_lookup(self.embeddings1, self.x)

        forward_cells = [rnn.BasicLSTMCell(self.num_hidden) for _ in range(self.num_layers)]
        bw_cells = [rnn.BasicLSTMCell(self.num_hidden) for _ in range(self.num_layers)]
        forward_cells = [rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob) for cell in forward_cells]
        bw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob) for cell in bw_cells]

        self.rnn_outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(
            forward_cells, bw_cells, self.x_emb, sequence_length=self.x_len, dtype=tf.float32)
        self.last_output = self.rnn_outputs[:, -1, :]
        self.logits = tf.contrib.slim.fully_connected(self.last_output, num_class, activation_fn=None)
        self.predictions = tf.argmax(self.logits, -1, output_type=tf.int32)
        self.loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss, global_step=self.global_step)
        # make prediction
        correct_predictions = tf.equal(self.predictions, self.y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [19]:
def batch_iter(inputs, outputs, batch_size, epochs_num):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    num_batches_per_epoch = (len(inputs) - 1) // batch_size + 1
    for epoch in range(epochs_num):
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, len(inputs))
            yield inputs[start_index:end_index], outputs[start_index:end_index]

In [20]:
num_class = 41

In [ ]:
tf.reset_default_graph()
with tf.Session() as sess:
    model = BidirectionalRNN(reversed_dictionary, doc_max_len, num_class)

    train_batches = batch_iter(train_x, train_y, 64, 10)
    num_batches_per_epoch = (len(train_x) - 1) // 64 + 1
    best_accuracy = 0
    
    sess.run(tf.global_variables_initializer())
    model_saver = tf.train.Saver(tf.global_variables())

    for x_batch, y_batch in train_batches:
        train_feed_dict = {
            model.x: x_batch,
            model.y: y_batch,
            model.keep_prob: 0.8
        }
        _, step, loss = sess.run([model.optimizer, model.global_step, model.loss], feed_dict=train_feed_dict)
        if step % 100 == 0:
            print("In step {0}, The loss is: {1}".format(step, loss))
        if step % 100 == 0:
            valid_batches = batch_iter(valid_x, valid_y, 64, 1)
            sum_accuracy, count = 0, 0
            for valid_x_batch, valid_y_batch in valid_batches:
                valid_feed_dict = {
                    model.x: valid_x_batch,
                    model.y: valid_y_batch,
                    model.keep_prob: 1.0
                }
                accuracy = sess.run(model.accuracy, feed_dict=valid_feed_dict)
                sum_accuracy += accuracy
                count += 1
            temp_accuracy = sum_accuracy / count

            print("\n Accuracy = {1}\n".format(step // num_batches_per_epoch, sum_accuracy / count))
            # Save the model that can produce the best accuracy
            if temp_accuracy > best_accuracy:
                best_accuracy = temp_accuracy
                model_saver.save(sess, "{0}/{1}.ckpt".format("saved_model", "naive"), global_step=step)
    
    print("end.")

Run BidirectionalRNN...
Loading Glove vectors...
start embedding...
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
In step 100, The loss is: 2.8512744903564453

 Accuracy = 0.308805910543131

In step 200, The loss is: 2.5423879623413086

 Accuracy = 0.35193690095846647

In step 300, The loss is: 2.3322501182556152

 Accuracy = 0.3878294728434505

In step 400, The loss is: 2.136857509613037

 Accuracy = 0.4020899893186344

In step 500, The loss is: 1.9058177471160889

 Accuracy = 0.4197117944304555

In step 600, The loss is: 2.4125397205352783

 Accuracy = 0.450063232130136

In step 700, The loss is: 2.0428404808044434

 Accuracy = 0.44866546855186124

In step 800, The loss is: 2.4209251403808594

 Accuracy = 0.4642738285727394

In step 900, The loss is: 2.0842390060424805

 Accuracy = 0.46315894568690097


 Accuracy = 0.5321319223211977

In step 10000, The loss is: 0.9677817821502686

 Accuracy = 0.5340288871774277

In step 10100, The loss is: 1.2003710269927979

 Accuracy = 0.5303347977205587

In step 10200, The loss is: 1.0953617095947266

 Accuracy = 0.528404552715655

In step 10300, The loss is: 1.4708642959594727

 Accuracy = 0.5294362354202392

In step 10400, The loss is: 0.788610577583313

 Accuracy = 0.5341287274330188

In step 10500, The loss is: 1.289271593093872

 Accuracy = 0.5310503194888179

In step 10600, The loss is: 1.114904522895813

 Accuracy = 0.5329472843450479

In step 10700, The loss is: 1.0345767736434937

 Accuracy = 0.5307008785942492

In step 10800, The loss is: 1.180433750152588

 Accuracy = 0.5284378328643287

In step 10900, The loss is: 0.8260524272918701

 Accuracy = 0.5274727102666618

In step 11000, The loss is: 1.2465569972991943

 Accuracy = 0.5275392705640092

In step 11100, The loss is: 1.0116686820983887

 Accuracy = 0.5314330405320603

In step 1120

In [19]:
class AttentionRNN(object):
    def __init__(self, reversed_dict, document_max_len, num_class):
        self.vocabulary_size = len(reversed_dict)
        self.embedding_size = 300
        self.num_hidden = 100
        self.num_layers = 2
        self.learning_rate = 1e-3

        self.x = tf.placeholder(tf.int32, [None, document_max_len], name="x")
        self.x_len = tf.reduce_sum(tf.sign(self.x), 1)
        self.y = tf.placeholder(tf.int32, [None], name="y")
        self.keep_prob = tf.placeholder(tf.float32, [], name="keep_prob")
        self.global_step = tf.Variable(0, trainable=False)

        with tf.name_scope("embedding"):
            init_embeddings = tf.random_uniform([self.vocabulary_size, self.embedding_size])
            self.embeddings = tf.get_variable("embeddings", initializer=init_embeddings, trainable=True)
            self.x_emb = tf.nn.embedding_lookup(self.embeddings, self.x)

        with tf.name_scope("birnn"):
            fw_cells = [rnn.BasicLSTMCell(self.num_hidden) for _ in range(self.num_layers)]
            bw_cells = [rnn.BasicLSTMCell(self.num_hidden) for _ in range(self.num_layers)]
            fw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob) for cell in fw_cells]
            bw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob) for cell in bw_cells]

            self.rnn_outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(
                fw_cells, bw_cells, self.x_emb, sequence_length=self.x_len, dtype=tf.float32)

        with tf.name_scope("attention"):
            self.attention_score = tf.nn.softmax(tf.contrib.slim.fully_connected(self.rnn_outputs, 1))
            self.attention_out = tf.squeeze(
                tf.matmul(tf.transpose(self.rnn_outputs, perm=[0, 2, 1]), self.attention_score),
                axis=-1)

        with tf.name_scope("output"):
            self.logits = tf.contrib.slim.fully_connected(self.attention_out, num_class, activation_fn=None)
            self.predictions = tf.argmax(self.logits, -1, output_type=tf.int32)

        with tf.name_scope("loss"):
            self.loss = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y))
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss, global_step=self.global_step)

        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, self.y)
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [20]:
tf.reset_default_graph()
with tf.Session() as sess:
    model = AttentionRNN(reversed_dictionary, doc_max_len, num_class)

    train_batches = batch_iter(train_x, train_y, 64, 10)
    num_batches_per_epoch = (len(train_x) - 1) // 64 + 1
    best_accuracy = 0
    
    sess.run(tf.global_variables_initializer())
    model_saver = tf.train.Saver(tf.global_variables())

    for x_batch, y_batch in train_batches:
        train_feed_dict = {
            model.x: x_batch,
            model.y: y_batch,
            model.keep_prob: 0.8
        }
        _, step, loss = sess.run([model.optimizer, model.global_step, model.loss], feed_dict=train_feed_dict)
        if step % 100 == 0:
            print("In step {0}, The loss is: {1}".format(step, loss))
        if step % 100 == 0:
            valid_batches = batch_iter(valid_x, valid_y, 64, 1)
            sum_accuracy, count = 0, 0
            for valid_x_batch, valid_y_batch in valid_batches:
                valid_feed_dict = {
                    model.x: valid_x_batch,
                    model.y: valid_y_batch,
                    model.keep_prob: 1.0
                }
                accuracy = sess.run(model.accuracy, feed_dict=valid_feed_dict)
                sum_accuracy += accuracy
                count += 1
            temp_accuracy = sum_accuracy / count

            print("\n Accuracy = {1}\n".format(step // num_batches_per_epoch, sum_accuracy / count))
            # Save the model that can produce the best accuracy
            if temp_accuracy > best_accuracy:
                best_accuracy = temp_accuracy
                model_saver.save(sess, "{0}/{1}.ckpt".format("saved_model", "naive"), global_step=step)
    
    print("end.")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
In step 100, The loss is: 3.4723939895629883

 Accuracy = 0.16480298190356824

In step 200, The loss is: 3.070613384246826

 Accuracy = 0.2612153886796567

In step 300, The loss is: 2.9416391849517822

 Accuracy = 0.2800685570072442

In step 400, The loss is: 2.58505916595459

 Accuracy = 0.316676650708095

In step 500, The loss is: 2.8270888328552246

 Accuracy = 0.3687433439512222

In step 600, The loss is: 2.4755616188049316

 Accuracy = 0.3902089989032989

In step 700, The loss is: 2.3452911376953125

 Accuracy = 0.4193956336845605

In step 800, The loss is: 2.3340704441070557

 Accuracy = 0.4456536209050078

In step 900, The loss is: 1.7437608242034912

 Accuracy = 0.42888045796571067

In step 1000, The loss is: 1.8474345207214355

 Accuracy = 0.4615448


 Accuracy = 0.5888744675503752

In step 10600, The loss is: 0.6057698726654053

 Accuracy = 0.5882754260168289

In step 10700, The loss is: 0.8657779693603516

 Accuracy = 0.5864616613418531

In step 10800, The loss is: 0.8532254695892334

 Accuracy = 0.5856462993180028

In step 10900, The loss is: 0.8311904072761536

 Accuracy = 0.5794728434504792

In step 11000, The loss is: 0.6224357485771179

 Accuracy = 0.5931343183921168

In step 11100, The loss is: 0.8200563788414001

 Accuracy = 0.5931343184873319

In step 11200, The loss is: 0.7442889213562012

 Accuracy = 0.5877928648132105

In step 11300, The loss is: 0.8108762502670288

 Accuracy = 0.5852302982213017

In step 11400, The loss is: 0.8627128005027771

 Accuracy = 0.5709864217252396

In step 11500, The loss is: 0.6276863813400269

 Accuracy = 0.5811701277955271

In step 11600, The loss is: 0.6953887939453125

 Accuracy = 0.5806376463688981

In step 11700, The loss is: 1.0666508674621582

 Accuracy = 0.5888744674551601

In step

In step 20800, The loss is: 0.3241724967956543

 Accuracy = 0.538754659243666

In step 20900, The loss is: 0.2669886648654938

 Accuracy = 0.5413671458681552

In step 21000, The loss is: 0.5064041614532471

 Accuracy = 0.5568756656334423

In step 21100, The loss is: 0.22831740975379944

 Accuracy = 0.5488052449096887

In step 21200, The loss is: 0.5905050039291382

 Accuracy = 0.5599540734824281

In step 21300, The loss is: 0.3659898638725281

 Accuracy = 0.5511015707882829

In step 21400, The loss is: 0.35174766182899475

 Accuracy = 0.5551284611605989

In step 21500, The loss is: 0.33016377687454224

 Accuracy = 0.5583732694673081

In step 21600, The loss is: 0.4601675868034363

 Accuracy = 0.560070553812356

In step 21700, The loss is: 0.1850145161151886

 Accuracy = 0.5353268104048964

In step 21800, The loss is: 0.2835187613964081

 Accuracy = 0.5622670394353593

In step 21900, The loss is: 0.3596092462539673

 Accuracy = 0.5607694356014934

In step 22000, The loss is: 0.288415431